In [1]:
from types import SimpleNamespace
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# Model

$$
\begin{aligned}
\max_{L_{W},L_{M},H_{W},H_{M}}&\frac{Q^{1-\rho}}{1-\rho}-\nu\left(\frac{T_{M}^{1+\epsilon}}{1+\epsilon}+\frac{T_{W}^{1+\epsilon}}{1+\epsilon}\right) \\
\text{s.t.}& \\
C&=w_{M}L_{M}+w_{W}L_{W} \\
H&=p_{M}H_{M}+p_{W}H_{W} \\
Q&=C^{\omega}H^{1-\omega}+\underline{Q} \\
X\in\{M,W\}:\,T_{X}&=\left(\left(L_{X}\right)^{\sigma}+\left(H_{X}\right)^{\sigma}\right)^{\frac{1}{\sigma}} \\
X\in\{M,W\}:L_{X}+H_{X}&\leq24 \\
X\in\{M,W\},Y\in{L,H}:Y_{X}&\in\left\{ 0,\frac{1}{2},1,\frac{3}{2},\dots24\right\} \\
\end{aligned}
$$

# Parameters

In [2]:
par = SimpleNamespace()

In [3]:
# a. wages
par.wM = 1.1
par.wW = 0.9

# b. productivity
par.pM = 0.9
par.pW = 1.1

# c. disutility of labor
par.nu = 0.0002
par.sigma = 2.0
par.epsilon = 1.0

# d. consumption utility
par.rho = 2.0
par.omega = 0.50
par.Qubar = 1e-8

# Solution functions

In [4]:
def utility_func(LM,HM,LW,HW,par):
    
    C = par.wM*LM + par.wW*LW
    H = par.pM*HM + par.pW*HW
    
    Q = C**par.omega+H**(1-par.omega)+par.Qubar
    
    utility = Q**(1-par.rho)/(1-par.rho) 
        
    disutility = par.nu*(LM**par.sigma+HM**par.sigma)**((1+par.epsilon)/par.sigma)
    disutility += par.nu*(LW**par.sigma+HW**par.sigma)**((1+par.epsilon)/par.sigma)
        
    return utility - disutility

In [5]:
def solve(par):
    
    # a. all possible choices
    x = np.linspace(0,24,49)
    LM,HM,LW,HW = np.meshgrid(x,x,x,x) # all combinations
    
    LM = LM.ravel() # vector
    HM = HM.ravel()
    LW = LW.ravel()
    HW = HW.ravel()

    # b. calculate utility
    u = utility_func(LM,HM,LW,HW,par)
    
    # c. set to minus infinity if constraint is broken
    I = (LM+HM > 24) | (LW+HW > 24) # | is "or"
    u[I] = -np.inf
    
    # d. find maximizing argument
    j = np.argmax(u)
    
    sol = {
        'LM':LM[j],
        'HM':HM[j],
        'LW':LW[j],
        'HW':HW[j]
    }
    
    return sol

# Solve

In [6]:
sol = solve(par)
print(sol)

{'LM': 7.0, 'HM': 6.0, 'LW': 6.0, 'HW': 7.0}


**Change parameters:**

In [7]:
# a. changes
par.wW = 1.1
par.wM = 0.9

par.pW = 0.9
par.pM = 1.1

# b. solution
sol = solve(par)
print(sol)

{'LM': 6.0, 'HM': 7.0, 'LW': 7.0, 'HW': 6.0}


# Export

In [12]:
K = 10

results = {}
results['LM'] = np.zeros(K)
results['HM'] = np.zeros(K)
results['LW'] = np.zeros(K)
results['HW'] = np.zeros(K)

wW_vec = np.linspace(0.9,1.1,K)

for i,wW in enumerate(wW_vec):
    
    print(f'{i} ',end='')
    
    par.wW = wW
    sol = solve(par)
    
    results['LM'][i] = sol['LM']
    results['HM'][i] = sol['HM']
    results['LW'][i] = sol['LW']
    results['HW'][i] = sol['HW']
    
pd.DataFrame(results).to_csv("results.csv")

0 1 2 3 4 5 6 7 8 9 